In [1]:
import json
import sys
import os
import logging
import argparse
from tqdm.auto import tqdm


/Users/yiqinhuang/anaconda3/envs/nlp_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
included_docs = set() 

file_path = os.path.join("v2_info_parsed", "combined_train_prompt1_v2.json")
with open(file_path, 'r') as file:
    articles = json.load(file)
    for article in articles:
        url = article['url']
        for doc in article['sources']:
            id = url + "#" + doc["Name"]
            included_docs.add(id)

file_path = os.path.join("v2_info_parsed", "combined_test_prompt1_v2.json")
with open(file_path, 'r') as file:
    articles = json.load(file)
    for article in articles:
        url = article['url']
        for doc in article['sources']:
            id = url + "#" + doc["Name"]
            included_docs.add(id)

print(f"A TOTAL OF {len(included_docs)} INCLUDED IN THE SEARCH")


A TOTAL OF 487422 INCLUDED IN THE SEARCH
